In [1]:
from keras.models import load_model
from keras.layers import Dense, Activation

In [16]:
import os
dataset_folder ='Datasets10/face_inputs/'
dir = os.listdir(dataset_folder)
labels = []
for item in dir:
    if os.path.isdir(dataset_folder+item) == True:
        labels.append(item)
labels

['.Uknown',
 'Azizi N',
 'Bhara M',
 'Cristiano Ronaldo',
 'Lionel Messi',
 'Tony_Blair']

In [17]:
model = load_model("model_batch_face.h5")

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 222, 222, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 109, 109, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0

In [19]:
model.layers

In [20]:
for i in range(len(model.layers)):
    if i > 24 :
        model.layers[i].trainable = True # dense layer
    else :
        model.layers[i].trainable = False

In [21]:
# model.compile(optimizer='adam', loss='categorical_crossentropy', 
#               metrics=['accuracy'])

In [22]:
for layer in model.layers:
    print(layer.trainable, '\t',layer.name)

False 	 conv2d
False 	 batch_normalization
False 	 max_pooling2d
False 	 dropout
False 	 conv2d_1
False 	 batch_normalization_1
False 	 max_pooling2d_1
False 	 dropout_1
False 	 conv2d_2
False 	 batch_normalization_2
False 	 max_pooling2d_2
False 	 dropout_2
False 	 conv2d_3
False 	 batch_normalization_3
False 	 max_pooling2d_3
False 	 dropout_3
False 	 conv2d_4
False 	 batch_normalization_4
False 	 max_pooling2d_4
False 	 dropout_4
False 	 conv2d_5
False 	 batch_normalization_5
False 	 max_pooling2d_5
False 	 dropout_5
False 	 flatten
True 	 dropout_6
True 	 dense
True 	 dense_1


In [23]:
model.pop()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 222, 222, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 109, 109, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0

In [24]:
model.add(Dense(len(labels),activation='softmax'))

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 222, 222, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 109, 109, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [27]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
detector_harcascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
detector_mtcnn = MTCNN()

In [28]:
def detect_face(img):
    pixels = np.asarray(img)
#     results = detector_harcascade.detectMultiScale(img,1.2,4)
    results = detector_mtcnn.detect_faces(pixels)
    if len(results) > 0:
        x1, y1, w, h = results[0]['box']
#         x1,y1,w,h = results[0]
        x1, y1 = abs(x1),abs(y1)
        x2,y2 = x1 + w, y1 + h
        img = pixels[y1:y2,x1:x2]
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img,(224,224))
        return img



def print_prog(val, val_len, folder, bar_size=20):
    progr = "="*round((val)*bar_size/val_len) + " "*round((val_len - (val))*bar_size/val_len)
    if val == 0:
        print("", end="\n")
    else:
        print("[%s] (%d samples) \t label : %s\t\t" % (progr, val+1,folder), end="\r")


# if use augmentasi image but is large for memory
# def img_augmentasi(img):
#     h,w = img.shape[:2]
#     center = (w//2,h//2)
    
#     M_rot5 = cv2.getRotationMatrix2D(center,5,1.0)
#     M_rot_neg_5 = cv2.getRotationMatrix2D(center,-5,1.0)
#     M_rot_10 = cv2.getRotationMatrix2D(center,10,1.0)
#     M_rot_neg_10 = cv2.getRotationMatrix2D(center,-10,1.0)
#     M_trans_3=np.float32([[1,0,3], [0,1,0]])
#     M_trans_neg_3=np.float32([[1,0,-3], [0,1,0]])
#     M_trans_6=np.float32([[1,0,6], [0,1,0]])
#     M_trans_neg_6=np.float32([[1,0,-6], [0,1,0]])
#     M_trans_y3=np.float32([[1,0,0], [0,1,3]])
#     M_trans_neg_y3=np.float32([[1,0,0], [0,1,-3]])
#     M_trans_y6=np.float32([[1,0,0], [0,1,6]])
#     M_trans_neg_y6=np.float32([[1,0,0], [0,1,-6]])
    
#     imgs=[]
#     imgs.append(cv2.warpAffine(img, M_rot5,(w,h),borderValue=(255,255,255)))
#     imgs.append(cv2.warpAffine(img, M_rot_neg_5,(w,h),borderValue=(255,255,255)))
#     imgs.append(cv2.warpAffine(img, M_rot_10,(w,h),borderValue=(255,255,255)))
#     imgs.append(cv2.warpAffine(img, M_rot_neg_10,(w,h),borderValue=(255,255,255)))
#     imgs.append(cv2.warpAffine(img, M_trans_3,(w,h),borderValue=(255,255,255)))
#     imgs.append(cv2.warpAffine(img, M_trans_neg_3,(w,h),borderValue=(255,255,255)))
#     imgs.append(cv2.warpAffine(img, M_trans_6,(w,h),borderValue=(255,255,255)))
#     imgs.append(cv2.warpAffine(img, M_trans_neg_6,(w,h),borderValue=(255,255,255)))
#     imgs.append(cv2.warpAffine(img, M_trans_y3,(w,h),borderValue=(255,255,255)))
#     imgs.append(cv2.warpAffine(img, M_trans_neg_y3,(w,h),borderValue=(255,255,255)))
#     imgs.append(cv2.warpAffine(img, M_trans_y6,(w,h),borderValue=(255,255,255)))
#     imgs.append(cv2.warpAffine(img, M_trans_neg_y6,(w,h),borderValue=(255,255,255)))
#     imgs.append(cv2.add(img, 10))
#     imgs.append(cv2.add(img, 30))
#     imgs.append(cv2.add(img, -10))
#     imgs.append(cv2.add(img, -30))
#     imgs.append(cv2.add(img, 15))
#     imgs.append(cv2.add(img, 45))
#     imgs.append(cv2.add(img, -15))
#     imgs.append(cv2.add(img, -45))
    
    
#     return imgs

In [29]:
data_sets = "Datasets10/face_inputs/"

names = []
images = []


for folder in os.listdir(data_sets):
    files = os.listdir(os.path.join(data_sets,folder))[:50]
    for i, name in enumerate(files):
        if (name.find(".jpg") > -1) or (name.find(".JPG") > -1):
            img = cv2.imread(os.path.join(data_sets + folder ,name))
            img = detect_face(img)
            if img is not None :
                images.append(img)
                names.append(folder)
                
                print_prog(i,len(files), folder)

[====================] (50 samples) 	 label : .Uknown		
[====================] (50 samples) 	 label : Azizi N		
[====================] (50 samples) 	 label : Bhara M		
[====================] (50 samples) 	 label : Cristiano Ronaldo		


In [46]:
# if use augmentasi data
# augmentasi_img = []
# augmentasi_names = []
# for i, img in enumerate(images):
#     try:
#         augmentasi_img.extend(img_augmentasi(img))
#         augmentasi_names.extend([names[i]] * 20)
#     except:
#         print(i)


# images.extend(augmentasi_img)
# names.extend(augmentasi_names)

In [55]:
len(names),len(images)

(5880, 5880)

In [56]:
unique , counts = np.unique(names,return_counts= True)
for item in zip(unique,counts):
    
    print(item)

('.Uknown', 1029)
('Azizi N', 1050)
('Bhara M', 1008)
('Cristiano Ronaldo', 924)
('Lionel Messi', 903)
('Tony_Blair', 966)


In [57]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(names)

labels = le.classes_

name_vec = le.transform(names)

categorical_name_vec = to_categorical(name_vec)

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
x_train, x_test, y_train, y_test = train_test_split(np.array(images, dtype=np.float32),   # input data
                                                    np.array(categorical_name_vec),       # target/output data 
                                                    test_size=0.15, 
                                                    random_state=42)

In [60]:
print(x_train.shape, y_train.shape, x_test.shape,  y_test.shape)

(4998, 224, 224) (4998, 6) (882, 224, 224) (882, 6)


In [61]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

In [62]:
EPOCHS = 10
BATCH_SIZE = 32
STEPS_PER_EPOCH = 16

history = model.fit(x_train, 
                    y_train,
                    epochs=EPOCHS,
                    
                    
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_split=0.2   # 15% of train dataset will be used as validation set
                    )

Epoch 1/10
16/16 [==============================] - 53s 3s/step - loss: 0.6136 - accuracy: 0.7599 - val_loss: 2.2924 - val_accuracy: 0.4660
Epoch 2/10
16/16 [==============================] - 56s 3s/step - loss: 0.5806 - accuracy: 0.7739 - val_loss: 2.5818 - val_accuracy: 0.4530
Epoch 3/10
16/16 [==============================] - 55s 3s/step - loss: 0.5821 - accuracy: 0.7719 - val_loss: 2.5751 - val_accuracy: 0.4540
Epoch 4/10
16/16 [==============================] - 53s 3s/step - loss: 0.5689 - accuracy: 0.7784 - val_loss: 2.3986 - val_accuracy: 0.4590
Epoch 5/10
16/16 [==============================] - 54s 3s/step - loss: 0.5764 - accuracy: 0.7711 - val_loss: 2.3725 - val_accuracy: 0.4640
Epoch 6/10
16/16 [==============================] - 57s 4s/step - loss: 0.5555 - accuracy: 0.7784 - val_loss: 2.2487 - val_accuracy: 0.4770
Epoch 7/10
16/16 [==============================] - 54s 3s/step - loss: 0.5707 - accuracy: 0.7721 - val_loss: 2.4999 - val_accuracy: 0.4590
Epoch 8/10
16/16 [==

In [ ]:
model.save("model_batch_face")